In [16]:
import pandas as pd
import re
# Function to clean player names
def clean_name(name):
    clean_name = re.split(r'\n', name)[0]
    return clean_name.strip()

def calculate_fantasy_score(row, match_difficulty=None):
    pos = row['pos']
    score = 0
    
    # Goals
    if 'FW' in pos:
        score += row['Performance Gls'] * 125000
    elif 'MF' in pos:
        score += row['Performance Gls'] * 150000
    elif 'DF' in pos:
        score += row['Performance Gls'] * 175000
    elif 'GK' in pos:
        score += row['Performance Gls'] * 250000
    
    # Assists
    score += row['Performance Ast'] * 60000
    
    # Yellow and Red Cards
    score -= row['Performance CrdY'] * 20000
    score -= row['Performance CrdR'] * 50000

    # Goalkeeper specific statistics
    if 'GK' in pos:
        score += row['saves'] * 5000  # Award points for saves
        score -= row['goals_against'] * 10000  # Deduct points for goals against
        score += row['save%'] * 1000  # Award points based on save percentage
    
    # Adjust score based on match difficulty if provided
    if match_difficulty:
        score *= match_difficulty.get(row['nation'], 1)
    
    # Weight score by the likelihood of starting (total minutes / 720)
    weight = row['minutes'] / 720
    print(row['name'], weight, row['minutes'])
    score *= weight
    
    return score

# Function to calculate multipliers from odds
def calculate_multipliers(odds):
    probabilities = {key: 1 / value for key, value in odds.items()}
    total = sum(probabilities.values())
    normalized_probabilities = {key: value / total for key, value in probabilities.items()}
    multipliers = {key: 1 / value for key, value in normalized_probabilities.items()}
    return multipliers

# Parse and store match odds
match_odds = {
    'GER': {'win': 1.30, 'draw': 6.00, 'lose': 12.50},
    'SCO': {'win': 12.50, 'draw': 6.00, 'lose': 1.30},
    'HUN': {'win': 3.50, 'draw': 3.50, 'lose': 2.20},
    'CHE': {'win': 2.20, 'draw': 3.50, 'lose': 3.50},
    'ESP': {'win': 1.78, 'draw': 3.85, 'lose': 5.00},
    'HRV': {'win': 5.00, 'draw': 3.85, 'lose': 1.78},
    'ITA': {'win': 1.39, 'draw': 4.85, 'lose': 9.70},
    'ALB': {'win': 9.70, 'draw': 4.85, 'lose': 1.39},
    'POL': {'win': 5.20, 'draw': 4.15, 'lose': 1.70},
    'NLD': {'win': 1.70, 'draw': 4.15, 'lose': 5.20},
    'SVN': {'win': 5.00, 'draw': 3.75, 'lose': 1.82},
    'DNK': {'win': 1.82, 'draw': 3.75, 'lose': 5.00},
    'SRB': {'win': 7.00, 'draw': 4.65, 'lose': 1.50},
    'ENG': {'win': 1.50, 'draw': 4.65, 'lose': 7.00},
    'ROU': {'win': 3.70, 'draw': 3.60, 'lose': 2.17},
    'UKR': {'win': 2.17, 'draw': 3.60, 'lose': 3.70},
    'BEL': {'win': 1.51, 'draw': 4.55, 'lose': 6.80},
    'SVK': {'win': 6.80, 'draw': 4.55, 'lose': 1.51},
    'AUT': {'win': 7.00, 'draw': 4.75, 'lose': 1.50},
    'FRA': {'win': 1.50, 'draw': 4.75, 'lose': 7.00},
    'TUR': {'win': 1.80, 'draw': 3.85, 'lose': 4.90},
    'GEO': {'win': 4.90, 'draw': 3.85, 'lose': 1.80},
    'PRT': {'win': 1.57, 'draw': 4.40, 'lose': 7.00},
    'CZE': {'win': 7.00, 'draw': 4.40, 'lose': 1.57}
}


In [17]:
# Calculate match difficulty multipliers
match_difficulty = {}
for team, odds in match_odds.items():
    multipliers = calculate_multipliers(odds)
    match_difficulty[team] = multipliers['win']

# Load the cleaned player prices
df_prices = pd.read_csv(r'data\cleaned_player_prices.csv')

# Apply the clean_name function to the name column
df_prices['name'] = df_prices['name'].apply(clean_name)

# Ensure names are standardized
df_prices['name'] = df_prices['name'].str.strip().str.title()

# Initialize FBRef data source for the Big 5 European Leagues combined
import soccerdata as sd
fbref = sd.FBref(leagues="Big 5 European Leagues Combined", seasons="2022-2023")

# Get player season statistics
player_season_stats = fbref.read_player_season_stats(stat_type="standard")

# Convert to DataFrame
df_stats = pd.DataFrame(player_season_stats)

# Reset the index to flatten the multi-level index
df_stats.reset_index(inplace=True)

# Flatten the MultiIndex columns
df_stats.columns = [' '.join(col).strip() for col in df_stats.columns.values]

# Rename columns to ensure clarity and consistency
df_stats.rename(columns={'player': 'name'}, inplace=True)

# Ensure names are standardized and matched correctly
df_stats['name'] = df_stats['name'].str.strip().str.title()

# Define the list of nationalities to include
nationalities_to_include = ['GER', 'ESP', 'ITA', 'NLD', 'ENG', 'BEL', 'FRA', 'PRT', 'DEN', 'TUR', 'UKR', 'CHE' ]
df_stats = df_stats[(df_stats['nation'].isin(nationalities_to_include))]

# Load Euro Qualifying data
df_qualifying = pd.read_csv(r'data\aggregated_player_stats.csv')

# Clean and standardize the names in the qualifying data
df_qualifying['name'] = df_qualifying['name'].apply(clean_name)
df_qualifying['name'] = df_qualifying['name'].str.strip().str.title()

# Merge qualifying data with the Big 5 data on player name and position
df_combined = pd.merge(df_stats, df_qualifying[['name', 'minutes', 'saves', 'goals_against', 'save%']], on='name', how='inner')

# Merge with price data
df_combined = pd.merge(df_combined, df_prices, on='name', how='inner')

# Enforce the rule that Antoine Griezmann is only considered a forward
df_combined.loc[df_combined['name'] == 'Antoine Griezmann', 'pos'] = 'FW'
df_combined.loc[df_combined['name'] == 'Kingsley Coman', 'pos'] = 'FW'
df_combined.loc[df_combined['name'] == 'Yannick Carrasco', 'pos'] = 'MF'
df_combined.loc[df_combined['name'] == 'Cengiz Ünder', 'pos'] = ''

# Update the fantasy score calculation to use the total minutes
df_combined['fantasy_score_with_difficulty'] = df_combined.apply(lambda row: calculate_fantasy_score(row, match_difficulty), axis=1)

# Calculate value for money (score) with match difficulty
df_combined['score_with_difficulty'] = df_combined['fantasy_score_with_difficulty'] / df_combined['Price']

# Function to select top players by position
def select_top_players(df, score_column):
    selected_team = pd.DataFrame()
    positions = {'GK': 1, 'DF': 3, 'MF': 4, 'FW': 3}
    
    for pos, count in positions.items():
        top_players = df[df['pos'].str.contains(pos)].sort_values(by=[score_column, 'minutes'], ascending=[False, False])
        selected_team = pd.concat([selected_team, top_players.head(count)])
        
        # Print the count of players for the current position
        print(f"Total players available for position {pos}: {df[df['pos'].str.contains(pos)].shape[0]}")
        print(f"Selected top {count} players for position {pos}:")
        print(top_players.head(count)[['name', 'score_with_difficulty']])
    
    # Ensure no more than 4 players from the same nation
    while selected_team['nation'].value_counts().max() > 4:
        for nation in selected_team['nation'].value_counts().index:
            if selected_team[selected_team['nation'] == nation].shape[0] > 4:
                lowest_scored_player = selected_team[selected_team['nation'] == nation].nsmallest(1, score_column)
                position = lowest_scored_player['pos'].values[0]
                next_best_player = df[(df['pos'].str.contains(position)) & (~df['name'].isin(selected_team['name'])) & (df['nation'] != nation)].nsmallest(1, 'Price')
                if next_best_player.empty:
                    continue
                selected_team = selected_team.drop(lowest_scored_player.index)
                selected_team = pd.concat([selected_team, next_best_player])

    return selected_team

# Example usage
selected_team_with_difficulty = select_top_players(df_combined, 'score_with_difficulty')

# Function to replace the lowest scored player with the next best cheaper player until the budget is within 50 million
def enforce_budget(selected_team, score_column, df):
    total_cost = selected_team['Price'].sum()
    
    while total_cost > 50_000_000:
        total_cost_millions = total_cost / 1_000_000
        print(f"Total cost is still over 50m, cost: {total_cost_millions:.2f}")
        
        # Find the lowest scored player
        sorted_team = selected_team.sort_values(by=score_column)
        
        for idx, lowest_scored_player in sorted_team.iterrows():
            print(f"Dropping lowest scored player: {lowest_scored_player['name']} with score {lowest_scored_player[score_column]} and price {lowest_scored_player['Price']}")
            
            position = lowest_scored_player['pos']
            
            # Find the next best cheaper player in the same position
            next_best_player = df[(df['pos'].str.contains(position)) & 
                                  (~df['name'].isin(selected_team['name'])) & 
                                  (df['Price'] < lowest_scored_player['Price'])].nlargest(1, score_column)
            
            if not next_best_player.empty:
                print(f"Adding next best player: {next_best_player['name'].values[0]} with score {next_best_player[score_column].values[0]} and price {next_best_player['Price'].values[0]}")
                
                selected_team = selected_team.drop(idx)
                selected_team = pd.concat([selected_team, next_best_player])
                total_cost = selected_team['Price'].sum()
                break
        else:
            # If no replacement found, attempt to drop the next lowest scored player
            print("No suitable replacement found with lower price and higher score. Trying next lowest scored player.")
            continue
        
        total_cost = selected_team['Price'].sum()
    
    return selected_team

# Example usage
selected_team_with_difficulty = select_top_players(df_combined, 'score_with_difficulty')

# Ensure at least 10 players for each position
def ensure_minimum_players(df, pos_column='pos', min_players=10):
    positions = ['GK', 'DF', 'MF', 'FW']
    additional_players = []

    for pos in positions:
        pos_players = df[df[pos_column].str.contains(pos)]
        if len(pos_players) < min_players:
            additional_needed = min_players - len(pos_players)
            additional_pos_players = df[~df.index.isin(pos_players.index) & df[pos_column].str.contains(pos)].head(additional_needed)
            additional_players.append(additional_pos_players)
    
    if additional_players:
        df = pd.concat([df] + additional_players).drop_duplicates()

    return df

# Ensure at least 10 players for each position
df_combined = ensure_minimum_players(df_combined)

# Select teams
selected_team_with_difficulty = select_top_players(df_combined, 'score_with_difficulty')

# Check the total cost for the team
total_cost_with_difficulty = selected_team_with_difficulty['Price'].sum()

# Enforce budget constraints
selected_team_with_difficulty = enforce_budget(selected_team_with_difficulty, 'score_with_difficulty', df_combined)

# Create the summary DataFrame
selected_team_summary = selected_team_with_difficulty[['name', 'Price', 'score_with_difficulty', 'pos']].sort_values(by=['score_with_difficulty'], ascending=[False])

# Save the summary to a new CSV file
selected_team_summary.to_csv(r'data\thewinners.csv', index=False)

# Display the summary
print("Selected Team with Difficulty:")
print(selected_team_summary)
print(f"Total Cost with Difficulty: {total_cost_with_difficulty:.2f} million")


[05/30/24 17:15:27] INFO     Saving cached data to C:\Users\Kasper\soccerdata\data\FBref              ]8;id=756555;file://c:\Users\Kasper\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=590248;file://c:\Users\Kasper\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#92\92]8;;\

Bukayo Saka 0.6805555555555556 490
Jorginho 0.2763888888888889 199
Leandro Trossard 0.40694444444444444 293
Oleksandr Zinchenko 0.8152777777777778 587
William Saliba 0.1638888888888889 118
Leander Dendoncker 0.11527777777777778 83
Ollie Watkins 0.07916666666666666 57
Illia Zabarnyi 1.125 810
Philip Billing 0.05555555555555555 40
Christian Nørgaard 0.5513888888888889 397
David Raya 0.25 180
Ivan Toney 0.013888888888888888 10
Mathias Jensen 0.42083333333333334 303
Leandro Trossard 0.40694444444444444 293
Conor Gallagher 0.13194444444444445 95
Jorginho 0.2763888888888889 199
Mykhailo Mudryk 0.9972222222222222 718
Joachim Andersen 0.875 630
Amadou Onana 0.48194444444444445 347
Jordan Pickford 2.0 1440
Vitaliy Mykolenko 1.1458333333333333 825
James Maddison 0.30277777777777776 218
Timothy Castagne 0.9652777777777778 695
Wout Faes 1.0 720
Youri Tielemans 0.5652777777777778 407
Trent Alexander-Arnold 0.49027777777777776 353
Aymeric Laporte 0.625 450
Cole Palmer 0.05138888888888889 37
Jack Gre

In [3]:
import pandas as pd
import re
import logging
from scipy.optimize import linprog

# Set up logging
logging.basicConfig(level=logging.INFO)

# Function to clean player names
def clean_name(name):
    return re.split(r'\n', name)[0].strip()

def calculate_fantasy_score(df, match_difficulty=None):
    score = pd.Series(0, index=df.index)
    
    # Goals
    score += df['Performance Gls'] * df['pos'].map({
        'FW': 125000, 
        'MF': 150000, 
        'DF': 175000, 
        'GK': 250000
    }).fillna(0)
    
    # Assists
    score += df['Performance Ast'] * 60000
    
    # Yellow and Red Cards
    score -= df['Performance CrdY'] * 20000
    score -= df['Performance CrdR'] * 50000

    # Goalkeeper specific statistics
    is_gk = df['pos'] == 'GK'
    score[is_gk] += df['saves'][is_gk] * 5000
    score[is_gk] -= df['goals_against'][is_gk] * 10000
    score[is_gk] += df['save%'][is_gk] * 1000
    
    # Adjust score based on match difficulty if provided
    if match_difficulty:
        score *= df['nation'].map(match_difficulty).fillna(1)
    
    # Weight score by the likelihood of starting (total minutes / 720)
    weight = df['minutes'] / 720
    score *= weight
    
    return score

# Function to calculate multipliers from odds
def calculate_multipliers(odds):
    probabilities = {key: 1 / value for key, value in odds.items()}
    total = sum(probabilities.values())
    normalized_probabilities = {key: value / total for key, value in probabilities.items()}
    return {key: 1 / value for key, value in normalized_probabilities.items()}

# Parse and store match odds
match_odds = {
    'GER': {'win': 1.30, 'draw': 6.00, 'lose': 12.50},
    'SCO': {'win': 12.50, 'draw': 6.00, 'lose': 1.30},
    'HUN': {'win': 3.50, 'draw': 3.50, 'lose': 2.20},
    'CHE': {'win': 2.20, 'draw': 3.50, 'lose': 3.50},
    'ESP': {'win': 1.78, 'draw': 3.85, 'lose': 5.00},
    'HRV': {'win': 5.00, 'draw': 3.85, 'lose': 1.78},
    'ITA': {'win': 1.39, 'draw': 4.85, 'lose': 9.70},
    'ALB': {'win': 9.70, 'draw': 4.85, 'lose': 1.39},
    'POL': {'win': 5.20, 'draw': 4.15, 'lose': 1.70},
    'NLD': {'win': 1.70, 'draw': 4.15, 'lose': 5.20},
    'SVN': {'win': 5.00, 'draw': 3.75, 'lose': 1.82},
    'DNK': {'win': 1.82, 'draw': 3.75, 'lose': 5.00},
    'SRB': {'win': 7.00, 'draw': 4.65, 'lose': 1.50},
    'ENG': {'win': 1.50, 'draw': 4.65, 'lose': 7.00},
    'ROU': {'win': 3.70, 'draw': 3.60, 'lose': 2.17},
    'UKR': {'win': 2.17, 'draw': 3.60, 'lose': 3.70},
    'BEL': {'win': 1.51, 'draw': 4.55, 'lose': 6.80},
    'SVK': {'win': 6.80, 'draw': 4.55, 'lose': 1.51},
    'AUT': {'win': 7.00, 'draw': 4.75, 'lose': 1.50},
    'FRA': {'win': 1.50, 'draw': 4.75, 'lose': 7.00},
    'TUR': {'win': 1.80, 'draw': 3.85, 'lose': 4.90},
    'GEO': {'win': 4.90, 'draw': 3.85, 'lose': 1.80},
    'PRT': {'win': 1.57, 'draw': 4.40, 'lose': 7.00},
    'CZE': {'win': 7.00, 'draw': 4.40, 'lose': 1.57}
}

# Calculate match difficulty multipliers
match_difficulty = {team: calculate_multipliers(odds)['win'] for team, odds in match_odds.items()}

# Load the cleaned player prices
df_prices = pd.read_csv(r'data\cleaned_player_prices.csv')
df_prices['name'] = df_prices['name'].apply(clean_name).str.strip().str.title()

# Initialize FBRef data source for the Big 5 European Leagues combined
import soccerdata as sd
fbref = sd.FBref(leagues="Big 5 European Leagues Combined", seasons="2023-2024")

# Get player season statistics
player_season_stats = fbref.read_player_season_stats(stat_type="standard")
df_stats = pd.DataFrame(player_season_stats).reset_index()
df_stats.columns = [' '.join(col).strip() for col in df_stats.columns.values]
df_stats.rename(columns={'player': 'name'}, inplace=True)
df_stats['name'] = df_stats['name'].str.strip().str.title()

# Define the list of nationalities to include
nationalities_to_include = ['GER', 'ESP', 'ITA', 'NLD', 'ENG', 'BEL', 'FRA', 'PRT', 'DEN', 'TUR', 'UKR', 'CHE']
df_stats = df_stats[df_stats['nation'].isin(nationalities_to_include)]

# Load Euro Qualifying data
df_qualifying = pd.read_csv(r'data\aggregated_player_stats.csv')
df_qualifying['name'] = df_qualifying['name'].apply(clean_name).str.strip().str.title()

# Merge dataframes
df_combined = pd.merge(df_stats, df_qualifying[['name', 'minutes', 'saves', 'goals_against', 'save%']], on='name', how='inner')
df_combined = pd.merge(df_combined, df_prices, on='name', how='inner')

# Enforce specific position rules
position_corrections = {
    'Antoine Griezmann': 'FW',
    'Kingsley Coman': 'FW',
    'Yannick Carrasco': 'MF',
    'Cengiz Ünder': '',
    'Oleksandr Zinchenko': 'MF'
}
df_combined.loc[df_combined['name'].isin(position_corrections.keys()), 'pos'] = df_combined['name'].map(position_corrections)

# Update the fantasy score calculation to use the total minutes
df_combined['fantasy_score_with_difficulty'] = calculate_fantasy_score(df_combined, match_difficulty)
df_combined['score_with_difficulty'] = df_combined['fantasy_score_with_difficulty'] / df_combined['Price']

# Define formations with their respective position requirements
formations = {
    '5-4-1': {'GK': 1, 'DF': 5, 'MF': 4, 'FW': 1},
    '5-3-2': {'GK': 1, 'DF': 5, 'MF': 3, 'FW': 2},
    '4-5-1': {'GK': 1, 'DF': 4, 'MF': 5, 'FW': 1},
    '4-4-2': {'GK': 1, 'DF': 4, 'MF': 4, 'FW': 2},
    '4-3-3': {'GK': 1, 'DF': 4, 'MF': 3, 'FW': 3},
    '3-5-2': {'GK': 1, 'DF': 3, 'MF': 5, 'FW': 2},
    '3-4-3': {'GK': 1, 'DF': 3, 'MF': 4, 'FW': 3}
}

# Function to enforce budget constraints using linear programming with positional constraints
def enforce_budget_lp(df, formation, budget=50000000):
    n = len(df)
    c = -df['score_with_difficulty'].values
    A_ub = [df['Price'].values]
    b_ub = [budget]
    A_eq = []
    b_eq = []
    
    # Adding constraints for positions based on the formation
    for pos, count in formation.items():
        pos_vector = df['pos'].str.contains(pos).astype(int).values
        A_eq.append(pos_vector)
        b_eq.append(count)

    bounds = [(0, 1)] * n

    # Linear programming to maximize the score under budget and position constraints
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

    if result.success:
        selected_indices = result.x.round().astype(bool)
        selected_team = df[selected_indices]
        return selected_team, -result.fun  # Return the selected team and the total score
    else:
        logging.warning("Linear programming optimization failed to find a valid solution.")
        return df.nlargest(11, 'score_with_difficulty'), df.nlargest(11, 'score_with_difficulty')['score_with_difficulty'].sum()

# Evaluate all formations and select the best one
best_team = None
best_score = -float('inf')
best_formation = None

for formation_name, formation in formations.items():
    selected_team, total_score = enforce_budget_lp(df_combined, formation)
    if total_score > best_score:
        best_score = total_score
        best_team = selected_team
        best_formation = formation_name

logging.info(f"Best Formation: {best_formation}")
logging.info(f"Best Team Score: {best_score}")

# Create the summary DataFrame for the best team
selected_team_summary = best_team[['name', 'Price', 'score_with_difficulty', 'pos']].sort_values(by=['score_with_difficulty'], ascending=[False])

# Save the summary to a new CSV file
selected_team_summary.to_csv('thewinners.csv', index=False)

# Display the summary
logging.info("Selected Team with Best Formation:")
logging.info(selected_team_summary)
total_cost_with_best_formation = best_team['Price'].sum()
logging.info(f"Total Cost with Best Formation: {total_cost_with_best_formation:.2f} million")


[06/03/24 11:10:58] INFO     No custom team name replacements found. You can configure these in       ]8;id=850579;file://c:\Python312\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=401069;file://c:\Python312\Lib\site-packages\soccerdata\_config.py#87\87]8;;\
                             C:\Users\khj.LASERTRYK\soccerdata\config\teamname_replacements.json.                  

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=87565;file://c:\Python312\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=525032;file://c:\Python312\Lib\site-packages\soccerdata\_config.py#182\182]8;;\
                             C:\Users\khj.LASERTRYK\soccerdata\config\league_dict.json.                            

                    INFO     Saving cached data to C:\Users\khj.LASERTRYK\soccerdata\data\FBref       ]8;id=282778;file://c:\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=333520;file://c:\Python312\Lib\site-packages\soccerdata\_common.py#92\92]8;;\

[06/03/24 11:11:09] INFO     Best Formation: 3-4-3                                                ]8;id=330642;file://C:\Users\khj.LASERTRYK\AppData\Local\Temp\ipykernel_62236\4089030590.py\4089030590.py]8;;\:]8;id=861604;file://C:\Users\khj.LASERTRYK\AppData\Local\Temp\ipykernel_62236\4089030590.py#177\177]8;;\

                    INFO     Best Team Score: 3.8646723898721596                                  ]8;id=234437;file://C:\Users\khj.LASERTRYK\AppData\Local\Temp\ipykernel_62236\4089030590.py\4089030590.py]8;;\:]8;id=924435;file://C:\Users\khj.LASERTRYK\AppData\Local\Temp\ipykernel_62236\4089030590.py#178\178]8;;\

                    INFO     Selected Team with Best Formation:                                   ]8;id=59775;file://C:\Users\khj.LASERTRYK\AppData\Local\Temp\ipykernel_62236\4089030590.py\4089030590.py]8;;\:]8;id=66357;file://C:\Users\khj.LASERTRYK\AppData\Local\Temp\ipykernel_62236\4089030590.py#187\187]8;;\

                    INFO                      name      Price  score_with_difficulty pos          ]8;id=126746;file://C:\Users\khj.LASERTRYK\AppData\Local\Temp\ipykernel_62236\4089030590.py\4089030590.py]8;;\:]8;id=339459;file://C:\Users\khj.LASERTRYK\AppData\Local\Temp\ipykernel_62236\4089030590.py#188\188]8;;\
                             50       Artem Dovbyk  5500000.0               1.019357  FW                           
                             73         Harry Kane  9000000.0               0.708832  FW                           
                             2         Declan Rice  4500000.0               0.425978  MF                           
                             81         Jonas Wind  3500000.0               0.411718  FW                           
                             23  Vitaliy Mykolenko  2500000.0               0.331126  DF                           
                             54    Jude Bellingham  8000000.0               0.258001  MF                           
                             92   Federico Dimarco  4500000.0               0.231096  DF                           
                             96      Adrien Rabiot  4000000.0               0.206178  MF                           
                             55       Mikel Merino  3500000.0               0.194894  MF                           
                             93   Francesco Acerbi  3000000.0               0.124176  DF                           
                             51       Andriy Lunin  3500000.0              -0.014074  GK                           

                    INFO     Total Cost with Best Formation: 51500000.00 million                  ]8;id=331633;file://C:\Users\khj.LASERTRYK\AppData\Local\Temp\ipykernel_62236\4089030590.py\4089030590.py]8;;\:]8;id=457821;file://C:\Users\khj.LASERTRYK\AppData\Local\Temp\ipykernel_62236\4089030590.py#190\190]8;;\

In [7]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Set up logging
logging.basicConfig(level=logging.INFO)

# Function to clean player names
def clean_name(name):
    return re.split(r'\n', name)[0].strip()

def calculate_fantasy_score(df, match_difficulty=None):
    score = pd.Series(0, index=df.index)
    
    # Goals
    score += df['Performance Gls'] * df['pos'].map({
        'FW': 125000, 
        'MF': 150000, 
        'DF': 175000, 
        'GK': 250000
    }).fillna(0)
    
    # Assists
    score += df['Performance Ast'] * 60000
    
    # Yellow and Red Cards
    score -= df['Performance CrdY'] * 20000
    score -= df['Performance CrdR'] * 50000

    # Goalkeeper specific statistics
    is_gk = df['pos'] == 'GK'
    score[is_gk] += df['saves'][is_gk] * 5000
    score[is_gk] -= df['goals_against'][is_gk] * 10000
    score[is_gk] += df['save%'][is_gk] * 1000
    
    # Adjust score based on match difficulty if provided
    if match_difficulty:
        score *= df['nation'].map(match_difficulty).fillna(1)
    
    # Weight score by the likelihood of starting (total minutes / 720)
    weight = df['minutes'] / 720
    score *= weight
    
    return score

# Function to calculate multipliers from odds
def calculate_multipliers(odds):
    probabilities = {key: 1 / value for key, value in odds.items()}
    total = sum(probabilities.values())
    normalized_probabilities = {key: value / total for key, value in probabilities.items()}
    return {key: 1 / value for key, value in normalized_probabilities.items()}

# Parse and store match odds
match_odds = {
    'GER': {'win': 1.30, 'draw': 6.00, 'lose': 12.50},
    'SCO': {'win': 12.50, 'draw': 6.00, 'lose': 1.30},
    'HUN': {'win': 3.50, 'draw': 3.50, 'lose': 2.20},
    'CHE': {'win': 2.20, 'draw': 3.50, 'lose': 3.50},
    'ESP': {'win': 1.78, 'draw': 3.85, 'lose': 5.00},
    'HRV': {'win': 5.00, 'draw': 3.85, 'lose': 1.78},
    'ITA': {'win': 1.39, 'draw': 4.85, 'lose': 9.70},
    'ALB': {'win': 9.70, 'draw': 4.85, 'lose': 1.39},
    'POL': {'win': 5.20, 'draw': 4.15, 'lose': 1.70},
    'NLD': {'win': 1.70, 'draw': 4.15, 'lose': 5.20},
    'SVN': {'win': 5.00, 'draw': 3.75, 'lose': 1.82},
    'DNK': {'win': 1.82, 'draw': 3.75, 'lose': 5.00},
    'SRB': {'win': 7.00, 'draw': 4.65, 'lose': 1.50},
    'ENG': {'win': 1.50, 'draw': 4.65, 'lose': 7.00},
    'ROU': {'win': 3.70, 'draw': 3.60, 'lose': 2.17},
    'UKR': {'win': 2.17, 'draw': 3.60, 'lose': 3.70},
    'BEL': {'win': 1.51, 'draw': 4.55, 'lose': 6.80},
    'SVK': {'win': 6.80, 'draw': 4.55, 'lose': 1.51},
    'AUT': {'win': 7.00, 'draw': 4.75, 'lose': 1.50},
    'FRA': {'win': 1.50, 'draw': 4.75, 'lose': 7.00},
    'TUR': {'win': 1.80, 'draw': 3.85, 'lose': 4.90},
    'GEO': {'win': 4.90, 'draw': 3.85, 'lose': 1.80},
    'PRT': {'win': 1.57, 'draw': 4.40, 'lose': 7.00},
    'CZE': {'win': 7.00, 'draw': 4.40, 'lose': 1.57}
}

# Calculate match difficulty multipliers
match_difficulty = {team: calculate_multipliers(odds)['win'] for team, odds in match_odds.items()}

# Load the cleaned player prices
df_prices = pd.read_csv(r'data\cleaned_player_prices.csv')
df_prices['name'] = df_prices['name'].apply(clean_name).str.strip().str.title()

# Initialize FBRef data source for the Big 5 European Leagues combined
import soccerdata as sd
fbref = sd.FBref(leagues="Big 5 European Leagues Combined", seasons="2023-2024")

# Get player season statistics
player_season_stats = fbref.read_player_season_stats(stat_type="standard")
df_stats = pd.DataFrame(player_season_stats).reset_index()
df_stats.columns = [' '.join(col).strip() for col in df_stats.columns.values]
df_stats.rename(columns={'player': 'name'}, inplace=True)
df_stats['name'] = df_stats['name'].str.strip().str.title()

# Define the list of nationalities to include
nationalities_to_include = ['GER', 'ESP', 'ITA', 'NLD', 'ENG', 'BEL', 'FRA', 'PRT', 'DEN', 'TUR', 'UKR', 'CHE']
df_stats = df_stats[df_stats['nation'].isin(nationalities_to_include)]

# Load Euro Qualifying data
df_qualifying = pd.read_csv(r'data\aggregated_player_stats.csv')
df_qualifying['name'] = df_qualifying['name'].apply(clean_name).str.strip().str.title()

# Merge dataframes
df_combined = pd.merge(df_stats, df_qualifying[['name', 'minutes', 'saves', 'goals_against', 'save%']], on='name', how='inner')
df_combined = pd.merge(df_combined, df_prices, on='name', how='inner')

# Enforce specific position rules
position_corrections = {
    'Antoine Griezmann': 'FW',
    'Kingsley Coman': 'FW',
    'Yannick Carrasco': 'MF',
    'Cengiz Ünder': '',
    'Oleksandr Zinchenko': 'MF'
}
df_combined.loc[df_combined['name'].isin(position_corrections.keys()), 'pos'] = df_combined['name'].map(position_corrections)

# Update the fantasy score calculation to use the total minutes
df_combined['fantasy_score_with_difficulty'] = calculate_fantasy_score(df_combined, match_difficulty)
df_combined['score_with_difficulty'] = df_combined['fantasy_score_with_difficulty'] / df_combined['Price']

# Define formations with their respective position requirements
formations = {
    '5-4-1': {'GK': 1, 'DF': 5, 'MF': 4, 'FW': 1},
    '5-3-2': {'GK': 1, 'DF': 5, 'MF': 3, 'FW': 2},
    '4-5-1': {'GK': 1, 'DF': 4, 'MF': 5, 'FW': 1},
    '4-4-2': {'GK': 1, 'DF': 4, 'MF': 4, 'FW': 2},
    '4-3-3': {'GK': 1, 'DF': 4, 'MF': 3, 'FW': 3},
    '3-5-2': {'GK': 1, 'DF': 3, 'MF': 5, 'FW': 2},
    '3-4-3': {'GK': 1, 'DF': 3, 'MF': 4, 'FW': 3}
}

# Calculate value per million
df_combined['value_per_million'] = df_combined['fantasy_score_with_difficulty'] / (df_combined['Price'] / 1_000_000)

# Optimization using Linear Programming
def optimize_team(df, budget=50_000_000):
    # Define the optimization problem
    prob = LpProblem("Fantasy_Team_Selection", LpMaximize)

    # Define decision variables
    player_vars = LpVariable.dicts("Player", df.index, cat='Binary')

    # Objective function: Maximize total fantasy score
    prob += lpSum([player_vars[i] * df.loc[i, 'fantasy_score_with_difficulty'] for i in df.index])

    # Constraint: Total cost should not exceed the budget
    prob += lpSum([player_vars[i] * df.loc[i, 'Price'] for i in df.index]) <= budget

    # Positional constraints
    positions = {'GK': 1, 'DF': 3, 'MF': 4, 'FW': 3}
    for pos, count in positions.items():
        prob += lpSum([player_vars[i] for i in df.index if pos in df.loc[i, 'pos']]) == count

    # Nationality constraint: No more than 4 players from the same nation
    for nation in df['nation'].unique():
        prob += lpSum([player_vars[i] for i in df.index if df.loc[i, 'nation'] == nation]) <= 4

    # Solve the optimization problem
    prob.solve()

    # Extract the selected team
    selected_team = df[[player_vars[i].varValue == 1 for i in df.index]]
    
    return selected_team

# Optimize team selection
optimized_team = optimize_team(df_combined)

# Create the summary DataFrame
optimized_team_summary = optimized_team[['name', 'Price', 'fantasy_score_with_difficulty', 'value_per_million', 'pos']]

# Save the summary to a new CSV file
optimized_team_summary.to_csv('optimized_team.csv', index=False)

# Display the summary
print("Optimized Team:")
print(optimized_team_summary)
print(f"Total Cost: {optimized_team['Price'].sum() / 1_000_000:.2f} million")
print(f"Total Fantasy Score: {optimized_team['fantasy_score_with_difficulty'].sum():.2f}")


[06/03/24 11:12:57] INFO     Saving cached data to C:\Users\khj.LASERTRYK\soccerdata\data\FBref       ]8;id=239263;file://c:\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=688109;file://c:\Python312\Lib\site-packages\soccerdata\_common.py#92\92]8;;\

Optimized Team:
                  name      Price  fantasy_score_with_difficulty  \
2          Declan Rice  4500000.0                 1916899.257552   
23   Vitaliy Mykolenko  2500000.0                  827815.549925   
43   Antoine Griezmann  7000000.0                 3217833.124478   
50        Artem Dovbyk  5500000.0                 5606462.529196   
51        Andriy Lunin  3500000.0                  -49258.445946   
55        Mikel Merino  3500000.0                  682129.170274   
73          Harry Kane  9000000.0                 6379489.567332   
88  Ruslan Malinovskyi  3000000.0                  441136.749249   
92    Federico Dimarco  4500000.0                 1039932.245132   
93    Francesco Acerbi  3000000.0                  372527.863688   
96       Adrien Rabiot  4000000.0                  824710.526316   

    value_per_million pos  
2       425977.612789  MF  
23       331126.21997  DF  
43      459690.446354  FW  
50      1019356.82349  FW  
51      -14073.841699  GK  